The following code will require the user to ctrl-a the webpage they view the list view of the course schedule on QUEST UWATERLOO and copy-paste the text onto a text file. After reading the text file, the program will output an ics file which would allow the user to upload the course dates+times onto a calendar app like Outlook.

In [1]:
import pandas as pd
import numpy as np
from icalendar import Calendar,Event
import uuid
from dateutil.rrule import rrule, WEEKLY
from icalendar.prop import vRecur

data = []
# Open the file in read mode
with open('texttoread.txt', 'r') as file:
    # Read each line in the file
    for line in file:
        if line.strip():
            data.append(line)

data = pd.DataFrame(data,columns=[0],dtype='string')

begin_chunk = data.index[ data.iloc[:,0].str.contains('Status') ].to_numpy() - 1
end_chunk = begin_chunk[1:]
last_elem = data.index[ data.iloc[:,0].str.contains('Printer Friendly Page') ].to_numpy()
end_chunk = np.append(end_chunk,last_elem[0])
chunk_indices = np.vstack((begin_chunk,end_chunk)).T

list_courses = {}
for chunk in chunk_indices:
    chunk_table = data.iloc[chunk[0]:chunk[1],:]
    chunk_table_name = chunk_table.iloc[0].to_string(index = False)
    chunk_table = chunk_table.drop(chunk_table.index[0])
    chunk_table = chunk_table.reset_index(drop=True)
    list_courses[chunk_table_name] = chunk_table

for key, course in list_courses.items():
    id = course.index[ course.iloc[:,0].str.contains('Class Nbr 	Section 	Component 	Days & Times 	Room 	Instructor 	Start/End Date') ].to_list()
    
    course = course.drop(index=list(range(0,id[0])))
    course = course.iloc[:,0].str.split(pat=' \t', expand=True)
    course.columns = course.iloc[0,:]
    course = course.drop(index=id[0])
    course = course.reset_index(drop=True)

    id1 = course.index[ course.iloc[:,0].str.contains(r'\d{2}/\d{2}/\d{4} - \d{2}/\d{2}/\d{4}') ].to_list()
    id1_len = len(id1)

    for i in range(6,-1,-1):
        course.iloc[0:id1_len,i] = course.iloc[id1,0]
        for j in id1:
            if j != 0:
                course.iloc[j, 0] = np.nan
        id1 = [x-1 if x-1 >= 0 else x for x in id1]

    course = course.dropna(how='all')
    course = course.ffill()


    list_courses[key] = course

df = pd.concat(list_courses)
df = df.reset_index()
df = df.drop(df.columns[1],axis=1)
df = df.rename(columns={df.columns[0]: "Course"})
df.columns = df.columns.str.replace('\n','')
df = df.map(lambda x: str(x).replace('\n', ''))

df = pd.concat([df, df['Start/End Date'].str.split(pat='-',expand=True)],axis=1)
df = df.drop('Start/End Date',axis=1)
df = df.rename(columns={df.columns[7]: "Start Date"})
df = df.rename(columns={df.columns[8]: "End Date"})

days_times = df['Days & Times'].str.split(pat=' ',expand=True)
days_times = days_times.rename(columns={days_times.columns[0]:'Days',days_times.columns[1]:'Start Time',days_times.columns[2]:'remove',days_times.columns[3]:'End Time'})
days_times = days_times.drop('remove',axis=1)
df = pd.concat([df,days_times],axis=1)
df = df.drop('Days & Times',axis=1)
df['Days'] = df['Days'].str.findall(r'[A-Z][^A-Z]*')

course_code_name = df['Course'].str.split(pat=' - ',expand=True)
course_code_name = course_code_name.rename(columns={course_code_name.columns[0]:'Course Code',course_code_name.columns[1]:'Course Name'})
df = pd.concat([course_code_name,df],axis=1)
df = df.drop('Course',axis=1)

df = df.astype( {'Course Code':'string','Course Name':'string','Class Nbr':'int32','Section':'string','Component':'string','Room':'string','Instructor':'string'} )
df['Start Date'] = pd.to_datetime(df['Start Date'].str.strip(), format='%m/%d/%Y')
df['Start Time'] = pd.to_datetime(df['Start Time'].str.strip(), format='%I:%M%p')
df['Start DateTime'] = df['Start Date'] + pd.to_timedelta(df['Start Time'].dt.strftime('%H:%M:%S'))

df['End Date'] = pd.to_datetime(df['End Date'].str.strip(), format='%m/%d/%Y')
df['End Time'] = pd.to_datetime(df['End Time'].str.strip(), format='%I:%M%p')
df['End DateTime'] = df['End Date'] + pd.to_timedelta(df['End Time'].dt.strftime('%H:%M:%S'))

day_map = {'M':0,'T':1,'W':2,'Th':3,'F':4,'S':5,'Su':6}
df = df.assign(Days=[[day_map[k] for k in row] for row in df.Days])

cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.dk//')
cal.add('version', '2.0')

for index, course in df.iterrows():
    event = Event()

    event.add('summary', course['Course Code']+' '+course['Component'])
    event.add('location', course['Room'])
    event.add('organizer',course['Instructor'])
    event.add('categories',course['Component'])
    event.add('description',course['Course Name'])

    event.add('dtstart',course['Start DateTime'])
    event.add('dtend',course['Start Date'] + pd.to_timedelta(course['End Time'].strftime('%H:%M:%S')))
    event.add('dtstamp',course['Start Date'])
    event.add('uid',str(uuid.uuid4()))
    
    rrule_string = str(rrule(freq=WEEKLY,dtstart=course['Start DateTime'],until=course['End DateTime'],byweekday=course['Days'])).split("\n")[1].replace('RRULE:','')
    rep_rule = vRecur.from_ical(rrule_string)
    event.add("RRULE", rep_rule)

    cal.add_component(event)

with open('schedule.ics', 'wb') as f:
    f.write(cal.to_ical())
    f.close()